# Day 9: 倒排索引

# 1. 預處理

今天的實作我們會使用華爾街日報的的文件集，我有預先將文件集切割成只有兩萬份文件的集合，這份文件集能夠從以下的code中下載。在今天的實作中，我們會將每一行視為一份文件來處理（利用NLTK工具來記號化和正規劃）。

In [1]:
import requests
from pathlib import Path

fname = 'wsta_col_20k.gz'
my_file = Path(fname)
if not my_file.is_file():
    url = 'https://hyhu.me/resources/' + fname
    r = requests.get(url)

    # Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)

來測試一下我們下載的結果，讀讀看第一行（第一份文件）。

In [2]:
import gzip

raw_docs = []
with gzip.open(fname, 'rt') as f:
    for raw_doc in f:
        raw_docs.append(raw_doc)

print(len(raw_docs))
print(raw_docs[0])

20000
John Blair & Co. is close to an agreement to sell its TV station advertising representation operation and program production unit to an investor group led by James H. Rosenfield, a former CBS Inc. executive, industry sources said. Industry sources put the value of the proposed acquisition at more than $100 million. John Blair was acquired last year by Reliance Capital Group Inc., which has been divesting itself of John Blair's major assets. John Blair represents about 130 local television stations in the placement of national and other advertising. Mr. Rosenfield stepped down as a senior executive vice president of CBS Broadcasting in December 1985 under a CBS early retirement program. Neither Mr. Rosenfield nor officials of John Blair could be reached for comment. 



接著，我們開始預處理。我們可以先用NLTK的工具`word_tokenize`來*記號化*每份文件，接著使用`PorterStemmer`來*stem*和*小寫化(lowercase)*各文件，並且把這些正規化後的字彙加上Unique ID存進Python的*dict*資料型態，所有的字彙$M$有自己的字彙ID$[0..M-1]$。這過程可能需要幾分鐘，如果一直沒跑出來不要緊張，等它一下～

In [3]:
import nltk

# processed_docs 儲存預處理過的文件列表
processed_docs = []
# vocab 儲存(term, term id)組合
vocab = {}
# total_tokens 儲存總共字數（不是字彙量，而是記號總量）
total_tokens = 0

# 使用PorterStemmer
stemmer = nltk.stem.PorterStemmer()

for raw_doc in raw_docs:
    
    # norm_doc 儲存正規化後的文件
    norm_doc = []
    
    # 使用word_tokenize
    tokenized_document = nltk.word_tokenize(raw_doc)
    for token in tokenized_document:
        stemmed_token = stemmer.stem(token).lower()
        norm_doc.append(stemmed_token)

        total_tokens += 1
        
        # 將正規化後的字彙存進vocab (不重複存同樣字型的字彙)
        if stemmed_token not in vocab:
            vocab[stemmed_token] = len(vocab)
            
    processed_docs.append(norm_doc)

print("Number of documents = {}".format(len(processed_docs)))
print("Number of unique terms = {}".format(len(vocab)))
print("Number of tokens = {}".format(total_tokens))

Number of documents = 20000
Number of unique terms = 103193
Number of tokens = 9140697


接著，我們可以使用Python的`Counter`來計算每個文件的詞頻。我們將每個字彙當作key，它的詞頻當作value。最後我們將所有文件各自的`Counter`存進*doc_term_freqs*列表中。

以一個Document為例：

>the old night keeper keeps the keep in the town. in the big old house in the big old gown. The house in the town had the big old keep where the old night keeper never did sleep. The keeper keeps the keep in the night and keeps in the dark and sleeps in the light.

在經過記號化和stemming之後，它的Counter應該長這樣：

`
Counter({'the': 14, 'in': 7, 'keep': 6, 'old': 5, '.': 4, 'night': 3, 'keeper': 3, 'big': 3, 'town': 2, 'hous': 2, 'sleep': 2, 'and': 2, 'gown': 1, 'had': 1, 'where': 1, 'never': 1, 'did': 1, 'dark': 1, 'light': 1})
`

In [4]:
from collections import Counter

# doc_term_freqs 儲存每個文件分別的字彙及詞頻Counter
doc_term_freqs = []

for norm_doc in processed_docs:
    tfs = Counter()
    # 計算詞頻
    for token in norm_doc:
        tfs[token] += 1
    doc_term_freqs.append(tfs)

print(len(doc_term_freqs))
print(doc_term_freqs[0])
print(doc_term_freqs[100])

20000
Counter({'.': 6, 'john': 5, 'blair': 5, 'of': 5, 'to': 3, 'rosenfield': 3, ',': 3, 'a': 3, 'cb': 3, 'the': 3, 'an': 2, 'station': 2, 'advertis': 2, 'and': 2, 'program': 2, 'group': 2, 'by': 2, 'inc.': 2, 'execut': 2, 'industri': 2, 'sourc': 2, 'in': 2, 'mr.': 2, '&': 1, 'co.': 1, 'is': 1, 'close': 1, 'agreement': 1, 'sell': 1, 'it': 1, 'tv': 1, 'represent': 1, 'oper': 1, 'product': 1, 'unit': 1, 'investor': 1, 'led': 1, 'jame': 1, 'h.': 1, 'former': 1, 'said': 1, 'put': 1, 'valu': 1, 'propos': 1, 'acquisit': 1, 'at': 1, 'more': 1, 'than': 1, '$': 1, '100': 1, 'million': 1, 'wa': 1, 'acquir': 1, 'last': 1, 'year': 1, 'relianc': 1, 'capit': 1, 'which': 1, 'ha': 1, 'been': 1, 'divest': 1, 'itself': 1, "'s": 1, 'major': 1, 'asset': 1, 'repres': 1, 'about': 1, '130': 1, 'local': 1, 'televis': 1, 'placement': 1, 'nation': 1, 'other': 1, 'step': 1, 'down': 1, 'as': 1, 'senior': 1, 'vice': 1, 'presid': 1, 'broadcast': 1, 'decemb': 1, '1985': 1, 'under': 1, 'earli': 1, 'retir': 1, 'neithe

# 2. 倒排索引

再來就是我們的倒排索引，開發上我主要分成六個部分：

1. 字彙表 `vocab` ，用以記錄字彙與其ID
2. 每個文件的長度 `doc_len`
3. `doc_ids` 是一個list，其中儲存了這個doc所包含的所有字彙的ID。is a list indexed by term IDs. For each term ID, it stores a list of document ids of all documents containing that term
4. `doc_term_freqs` 是一個list，其中儲存了這個doc中相對應`doc_ids`的詞頻（就像Day 7中的第二個倒排索引）。每一個term ID都應該儲存著自己的文件-字彙詞頻列表(document term frequencies $f_{d,t}$)，這個列表說明了每個檔案 $d$ 中各個字彙 $t$ 分別出現的頻率 $f$ 
5. `doc_term_freqs` 記錄了各個詞出現在每個文件的詞頻， 而 `doc_freqs` 則記錄各個詞總共出現在多少個文件中。這會跟我們明天所要說的TF-IDF有關。文件頻率(Document Frequency) $f_t$ 的記法是，只要他曾經出現過，$f_t$ 就會+1。
6. 最後我存了兩個數字 `total_num_docs` 和 `max_doc_len` ，他們分別記錄了總共處理的文件數量（應該要是兩萬份）以及單一文件最長的長度

我所存的一些資料並不是為了之後計算TF-IDF用的，而是一些方便我們驗證開發正確性的統計數字。

In [5]:
class InvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1

    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        return self.doc_ids[term_id]

    def freqs(self, term):
        term_id = self.vocab[term]
        return self.doc_term_freqs[term_id]

    def f_t(self, term): 
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

    def space_in_bytes(self):
        # 我們假設每個integer使用8 bytes
        space_usage = 0
        for doc_list in self.doc_ids:
            space_usage += len(doc_list) * 8
        for freq_list in self.doc_term_freqs:
            space_usage += len(freq_list) * 8
        return space_usage
    

invindex = InvertedIndex(vocab, doc_term_freqs)

# print inverted index stats
print("documents = {}".format(invindex.num_docs()))
print("number of terms = {}".format(invindex.num_terms()))
print("longest document length = {}".format(invindex.max_doc_len))
print("uncompressed space usage MiB = {:.3f}".format(invindex.space_in_bytes() / (1024.0 * 1024.0)))

documents = 20000
number of terms = 103193
longest document length = 10576
uncompressed space usage MiB = 58.187


# Day 10: TF-IDF

我們現在根據Day 9中開發的 `InvertedIndex` 類別來計算文件和查詢 $Q$ 之間的TF-IDF相似度。

在這裡，我們使用簡易版的TF-IDF相似度計算公式：

\begin{equation*}
Score(Q,d) = \frac{1}{\sqrt{|d|}} \times \sum_{i=1}^q \log(1 + f_{d,t}) * \log( \frac{N}{f_t} ) 
\end{equation*}

其中 $Q$ 指的是我們的查詢，包含了多個查詢字 $q$ 。 $\sqrt{|d|}$ 說明了文件的長度， $f_{d,t}$ 是字詞 $t$ 在文件 $d$ 中的出現頻率， $N$ 是指在文件集中的文件總數， 最後 $f_t$ 為包含了字詞 $t$ 的文件數量。這些資料都是我們在 `InvertedIndex` 類別中預先存好的。

我們的 `query_tfidf` function中輸入一句查詢以及一個索引類別，最後回傳 top-$k$ TF-IDF分數最高的文件。

In [6]:
from math import log, sqrt

# 給定一個查詢(String)和一個索引(Class)，回傳k個文件
def query_tfidf(query, index, k=10):
    
    # scores 儲存了docID和他們的TF-IDF分數
    scores = Counter()
    
    N = index.num_docs()
    
    for term in query:
        i = 0
        f_t = index.f_t(term)
        for docid in index.docids(term):
            # f_(d,t)
            f_d_t = index.freqs(term)[i]
            d = index.doc_len[docid]
            tfidf_cal = log(1+f_d_t) * log(N/f_t) / sqrt(d)
            scores[docid] += tfidf_cal
            i += 1
    
    return scores.most_common(k)

# 查詢語句
query = "south korea production"
# 預處理查詢，為了讓查詢跟索引內容相同
stemmed_query = nltk.stem.PorterStemmer().stem(query).split()
results = query_tfidf(stemmed_query, invindex)
for rank, res in enumerate(results):
    # e.g 排名 1 DOCID 176 SCORE 0.426 內容 South Korea rose 1% in February from a year earlier, the
    print("排名 {:2d} DOCID {:8d} SCORE {:.3f} 內容 {:}".format(rank+1,res[0],res[1],raw_docs[res[0]][:75]))

排名  1 DOCID     1084 SCORE 1.210 內容 Unemployment in South Korea fell to 3.8% of the labor force last year from 
排名  2 DOCID      905 SCORE 1.127 內容 Seasonally adjusted industrial production in South Korea increased nearly 2
排名  3 DOCID     5612 SCORE 1.056 內容 Consumer prices in South Korea rose 2.4% in April from a year-earlier and 0
排名  4 DOCID     9126 SCORE 0.998 內容 Foreign investment in South Korea totaled $278 million in 1987's first quar
排名  5 DOCID    17960 SCORE 0.936 內容 Henley Group Inc. said its M.W. Kellogg Co. unit received a contract to des
排名  6 DOCID     1760 SCORE 0.926 內容 Consumer prices in South Korea rose 1% in February from a year earlier, the
排名  7 DOCID     4132 SCORE 0.926 內容 South Korea's trade deficit with Japan grew to a record $629 million in Apr
排名  8 DOCID    17826 SCORE 0.923 內容 South Korea revised its 1986 current-account surplus to $5 billion from the
排名  9 DOCID    15803 SCORE 0.911 內容 South Korea's 1986 trade surplus with the U.S. was revised upward to

# Day 12: Vbyte 壓縮

今天我們要來實作倒排索引的空間壓縮。這裡我們會利用昨天文中介紹的VByte壓縮法壓縮倒排索引中的文件ID `doc_ids` 以及文件-詞頻列表 `doc_term_freqs` 。

在昨天的文中，我們有附上Vbyte壓縮和解壓縮的演算法，我們將開發成以下兩個方法：

- `vbyte_encode(num)`：接受一個數字，回傳相對該數字的Vbyte壓縮。
- `vbyte_decode(input_bytes, idx)`：接受一組Vbyte壓縮（可能是多個位元組，根據Continuation Bit來決定有幾個bytes），回傳解壓縮後的數字。

In [7]:
def vbyte_encode(num):

    # out_bytes 儲存轉換成Vbyte壓縮後的格式
    out_bytes = []
    
    while num >= 128:
        out_bytes.append(int(num) % 128)
        num /= 128
        
    out_bytes.append(int(num) + 128)
    
    return out_bytes


def vbyte_decode(input_bytes, idx):
    
    x = 0 # 儲存解壓縮後的數字
    s = 0
    consumed = 0 # 記錄花了多少位元組來解壓這個數字
    
    while input_bytes[idx + consumed] < 128:
        x ^= (input_bytes[idx + consumed] << s)
        s += 7
        consumed += 1
    
    x ^= ((input_bytes[idx + consumed]-128) << s)
    consumed += 1
    
    return x, consumed


單元測試壓縮和解壓縮過程正確性：

In [8]:
for num in range(0, 123456):
    vb = vbyte_encode(num)
    dec, decoded_bytes = vbyte_decode(vb, 0)
    assert(num == dec)
    assert(decoded_bytes == len(vb))

正確地開發了VByte壓縮和解壓縮之後，我們來修正原本的 `InvertedIndex` 類別以支援VByte壓縮。需要注意的是， `doc_ids` 的部份是要壓縮文件ID之間的間隔而不是文件ID本身。我還寫了一個輔助方法 `decompress_list` 來幫助我們更簡單地將列表解壓縮。

In [12]:
def decompress_list(input_bytes, gapped_encoded):
    res = []
    prev = 0
    idx = 0
    while idx < len(input_bytes):
        dec_num, consumed_bytes = vbyte_decode(input_bytes, idx)
        idx += consumed_bytes
        num = dec_num + prev
        res.append(num)
        if gapped_encoded:
            prev = num
    return res

class CompressedInvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1
        
        # 壓縮文件ID之間的間隔
        for i in range(len(self.doc_ids)):
            last_docid = self.doc_ids[i][0]
            for j in range(len(self.doc_ids[i])):
                if j != 0:
                    ori_docid = self.doc_ids[i][j]
                    self.doc_ids[i][j] = vbyte_encode(self.doc_ids[i][j] - last_docid)
                    last_docid = ori_docid
                else:
                    self.doc_ids[i][0] = vbyte_encode(last_docid)
            self.doc_ids[i] = sum(self.doc_ids[i], [])
            
        # 根據詞頻壓縮
        for i in range(len(self.doc_term_freqs)):
            for j in range(len(self.doc_term_freqs[i])):
                self.doc_term_freqs[i][j] = vbyte_encode(self.doc_term_freqs[i][j])
            self.doc_term_freqs[i] = sum(self.doc_term_freqs[i], [])
    
    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        # 解壓縮
        return decompress_list(self.doc_ids[term_id], True)

    def freqs(self, term):
        term_id = self.vocab[term]
        # 解壓縮
        return decompress_list(self.doc_term_freqs[term_id], False)

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

    def space_in_bytes(self):
        # 這裡現在假設數字都是位元組型態
        space_usage = 0
        for doc_list in self.doc_ids:
            space_usage += len(doc_list)
        for freq_list in self.doc_term_freqs:
            space_usage += len(freq_list)
        return space_usage


compressed_index = CompressedInvertedIndex(vocab, doc_term_freqs)

print("documents = {}".format(compressed_index.num_docs()))
print("unique terms = {}".format(compressed_index.num_terms()))
print("longest document = {}".format(compressed_index.max_doc_len))
print("compressed space usage MiB = {:.3f}".format(compressed_index.space_in_bytes() / (1024.0 * 1024.0)))

documents = 20000
unique terms = 103193
longest document = 10576
compressed space usage MiB = 7.818


在資料不變的情況下，我們的空間使用已經從58.187MiB降到了7.818MiB。

最後，我們來測試原本的倒排索引及VByte壓縮後的倒排索引結果有沒有改變（理論上結果該相同）。

In [13]:
# 確認是否和先前結果相同
query = "south korea production"
stemmed_query = nltk.stem.PorterStemmer().stem(query).split()
comp_results = query_tfidf(stemmed_query, compressed_index)
for rank, res in enumerate(comp_results):
    print("排名 {:2d} DOCID {:8d} SCORE {:.3f} 內容 {:}".format(rank+1,res[0],res[1],raw_docs[res[0]][:75]))

排名  1 DOCID     1084 SCORE 1.210 內容 Unemployment in South Korea fell to 3.8% of the labor force last year from 
排名  2 DOCID      905 SCORE 1.127 內容 Seasonally adjusted industrial production in South Korea increased nearly 2
排名  3 DOCID     5612 SCORE 1.056 內容 Consumer prices in South Korea rose 2.4% in April from a year-earlier and 0
排名  4 DOCID     9126 SCORE 0.998 內容 Foreign investment in South Korea totaled $278 million in 1987's first quar
排名  5 DOCID    17960 SCORE 0.936 內容 Henley Group Inc. said its M.W. Kellogg Co. unit received a contract to des
排名  6 DOCID     1760 SCORE 0.926 內容 Consumer prices in South Korea rose 1% in February from a year earlier, the
排名  7 DOCID     4132 SCORE 0.926 內容 South Korea's trade deficit with Japan grew to a record $629 million in Apr
排名  8 DOCID    17826 SCORE 0.923 內容 South Korea revised its 1986 current-account surplus to $5 billion from the
排名  9 DOCID    15803 SCORE 0.911 內容 South Korea's 1986 trade surplus with the U.S. was revised upward to

不論DocID排名或是TF-IDF分數都沒有改變，壓縮結果正確。